In [38]:
import torch 

# Tensor simple operations

In [39]:
tensor2d = torch.tensor([[2, 3, 7], [3, 4, 8]])
tensor2d

tensor([[2, 3, 7],
        [3, 4, 8]])

In [40]:
tensor2d.shape

torch.Size([2, 3])

In [41]:
# reshape tensors
tensor2d_reshape = tensor2d.reshape(3, 2)
tensor2d_reshape

tensor([[2, 3],
        [7, 3],
        [4, 8]])

In [42]:
# Transposing
tensor2d.T

tensor([[2, 3],
        [3, 4],
        [7, 8]])

In [43]:
# Matrix Multiplication (2 methods)
tensor2d.matmul(tensor2d_reshape)
tensor2d @ tensor2d_reshape

tensor([[53, 71],
        [66, 85]])

# Models on computation graphs

## Simple Logistic Regression using Neural Nets

In [44]:
import torch.nn.functional as F
from torch.autograd import grad

### Without Backpropagation

In [46]:
true = torch.tensor([1.0], dtype=torch.float32) # true labels

# Network
inputs = torch.tensor([1.1], dtype=torch.float32) # input feature
weights = torch.tensor([2.2], dtype=torch.float32) # weight params
bias = torch.tensor([0.0], dtype=torch.float32) # bias unit
function_output = inputs * weights + bias
activation_output = torch.sigmoid(function_output)

# Check loss after output()
loss_output = F.binary_cross_entropy(activation_output, true)
loss_output

tensor(0.0852)

### With Backpropagation

In [53]:
true = torch.tensor([1.0], dtype=torch.float32) # true labels

# Network
inputs = torch.tensor([1.1], dtype=torch.float32) # input feature
weights = torch.tensor([2.2], dtype=torch.float32, requires_grad=True) # weight params
bias = torch.tensor([0.0], dtype=torch.float32, requires_grad=True) # bias unit
function_output = inputs * weights + bias
activation_output = torch.sigmoid(function_output)

# Check loss after output()
loss_output = F.binary_cross_entropy(activation_output, true)

# apply backpropagation using comparing true and activation output.
grad_L_weights = grad(loss_output, weights, retain_graph=True)
grad_L_bias = grad(loss_output, bias, retain_graph=True)

print(grad_L_weights)
print(grad_L_bias)
print(loss_output)

# or instaed of typing the above code to output the grad_L outputs, we can use the following
loss_output.backward()
print(weights.grad)
print(bias.grad)
print(loss_output)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([-0.0898])
tensor([-0.0817])
tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)


### Apply forward propagation after using the new weight and bias params

In [57]:
true = torch.tensor([1.0], dtype=torch.float32) # true labels

# Network
inputs = torch.tensor([1.1], dtype=torch.float32) # input feature
weights = torch.tensor([-0.8462], dtype=torch.float32, requires_grad=True) # weight params
bias = torch.tensor([-0.7693], dtype=torch.float32, requires_grad=True) # bias unit
function_output = inputs * weights + bias
activation_output = torch.sigmoid(function_output)

# Check loss after output()
loss_output = F.binary_cross_entropy(activation_output, true)

# apply backpropagation using comparing true and activation output.
grad_L_weights = grad(loss_output, weights, retain_graph=True)
grad_L_bias = grad(loss_output, bias, retain_graph=True)

print(grad_L_weights)
print(grad_L_bias)
print(loss_output)

# or instaed of typing the above code to output the grad_L outputs, we can use the following
loss_output.backward()
print(weights.grad)
print(bias.grad)
print(loss_output)

(tensor([-0.9301]),)
(tensor([-0.8456]),)
tensor(1.8679, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([-0.9301])
tensor([-0.8456])
tensor(1.8679, grad_fn=<BinaryCrossEntropyBackward0>)
